In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFUR5A7/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFUR5A7/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFUR5A7/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
import numpy as np
train, test = imdb['train'], imdb['test']

In [ ]:
training_sentences = []
training_labels = []


test_sentences = []
test_labels = []

for s,l in train:
    training_sentences.append(s.numpy().decode('utf-8'))
    training_labels.append(l.numpy())
for s,l in test:
    test_sentences.append(s.numpy().decode('utf-8'))
    test_labels.append(l.numpy())

In [ ]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(test_labels)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
train_tokenizer.fit_on_texts(training_sentences)
train_sequences = train_tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(train_sequences, maxlen=120, padding='post')

In [ ]:
test_tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
test_tokenizer.fit_on_texts(test_sentences)
test_sequences = test_tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=120, padding='post')

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(10000, 16, input_length=120),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(6, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(padded, training_labels_final, validation_data=(test_padded, testing_labels_final),
          epochs=10 )

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4459 - accuracy: 0.7795 - val_loss: 0.9270 - val_accuracy: 0.5268
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1987 - accuracy: 0.9251 - val_loss: 1.0917 - val_accuracy: 0.5266
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0695 - accuracy: 0.9841 - val_loss: 1.4260 - val_accuracy: 0.5248
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0164 - accuracy: 0.9979 - val_loss: 1.6973 - val_accuracy: 0.5230
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0044 - accuracy: 0.9998 - val_loss: 1.9302 - val_accuracy: 0.5238
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 2.1117 - val_accuracy: 0.5244
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 7.3808e-04 - accuracy: 1.0000 - val_loss: 2.2732 - val_accuracy: 0.5256
Ep

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
word_index = train_tokenizer.word_index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, 10000):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sentence = "I really think this is amazing. honest."
sequence = train_tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
